In [69]:
# import numpy as np
import pickle

class Agent():
    
    def __init__(self,move_value):
        self.move_value = move_value
        self.value = dict()
    
    def state(self,current_state):
        if str(current_state) in self.value:
            
        else: 
            self.value[str(current_state)] = 0

    def action(self):
        pass
    def reward(self):
        pass

class Environment():
    
    def __init__(self):
        self.board = np.zeros((3,3))
    
    # Def: Check if there is any winner
    def check_winner(self):
        for row in self.board:
            if sum(row) == 3: return 1
            if sum(row) ==-3: return -1 
            
        for row in np.transpose(self.board):
            if sum(row) == 3: return 1
            if sum(row) ==-3: return -1 
            
        if sum([self.board[i][i] for i in range(len(self.board))]) == 3:  return 1
        if sum([self.board[i][i] for i in range(len(self.board))]) == -3: return -1
        if sum([self.board[i][len(self.board)-i-1] for i in range(len(self.board))]) == 3: return  1       
        if sum([self.board[i][len(self.board)-i-1] for i in range(len(self.board))]) == -3:return -1        
            
    # Def: Check if there is a winner
    def check_if_there_is_winner(self): return self.check_winner()
    
    # change the borad position - location and value.      
    def make_a_move(self,position,value):
        assert (value == 1 or value == -1)
        assert (self.board.ravel()[position] == 0)
        self.board.ravel()[position] = value
    
    # Reset the board back the starting position.     
    def reset_game(self): self.board = np.zeros((3,3))
        
    # View the current state of the board
    def get_board(self): return self.board

In [70]:
temp = Environment()
temp.make_a_move(0,1)
temp.make_a_move(1,1)
temp.make_a_move(8,-1)
print(temp.get_board())
print(temp.check_if_there_is_winner())
temp.make_a_move(2,1)
print(temp.get_board())
print(temp.check_if_there_is_winner())


[[ 1.  1.  0.]
 [ 0.  0.  0.]
 [ 0.  0. -1.]]
None
[[ 1.  1.  1.]
 [ 0.  0.  0.]
 [ 0.  0. -1.]]
1


In [62]:
temp_dict = dict()
temp_dict[str(temp.get_board())] = 4
temp_dict

{'[[ 1.  1.  1.]\n [ 0.  0.  0.]\n [ 0.  0. -1.]]': 4}

In [71]:
temp.make_a_move(4,1)
temp.make_a_move(4,1)

AssertionError: 

In [72]:
temp.get_board()

array([[ 1.,  1.,  1.],
       [ 0.,  1.,  0.],
       [ 0.,  0., -1.]])

In [73]:
! git all-go

[master d4f89dc] commit
 2 files changed, 164 insertions(+), 32 deletions(-)
Counting objects: 4, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 1.51 KiB | 1.51 MiB/s, done.
Total 4 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/JaeDukSeo/tictactoe.git
   fea02f9..d4f89dc  master -> master


In [74]:
import numpy as np
import pickle

BOARD_ROWS = 3
BOARD_COLS = 3
BOARD_SIZE = BOARD_ROWS * BOARD_COLS

class State:
    def __init__(self):
        # the board is represented by an n * n array,
        # 1 represents a chessman of the player who moves first,
        # -1 represents a chessman of another player
        # 0 represents an empty position
        self.data = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.winner = None
        self.hash_val = None
        self.end = None

    # compute the hash value for one state, it's unique
    def hash(self):
        if self.hash_val is None:
            self.hash_val = 0
            for i in self.data.reshape(BOARD_ROWS * BOARD_COLS):
                if i == -1:
                    i = 2
                self.hash_val = self.hash_val * 3 + i
        return int(self.hash_val)

    # check whether a player has won the game, or it's a tie
    def is_end(self):
        if self.end is not None:
            return self.end
        results = []
        # check row
        for i in range(0, BOARD_ROWS):
            results.append(np.sum(self.data[i, :]))
        # check columns
        for i in range(0, BOARD_COLS):
            results.append(np.sum(self.data[:, i]))

        # check diagonals
        results.append(0)
        for i in range(0, BOARD_ROWS):
            results[-1] += self.data[i, i]
        results.append(0)
        for i in range(0, BOARD_ROWS):
            results[-1] += self.data[i, BOARD_ROWS - 1 - i]

        for result in results:
            if result == 3:
                self.winner = 1
                self.end = True
                return self.end
            if result == -3:
                self.winner = -1
                self.end = True
                return self.end

        # whether it's a tie
        sum = np.sum(np.abs(self.data))
        if sum == BOARD_ROWS * BOARD_COLS:
            self.winner = 0
            self.end = True
            return self.end

        # game is still going on
        self.end = False
        return self.end

    # @symbol: 1 or -1
    # put chessman symbol in position (i, j)
    def next_state(self, i, j, symbol):
        new_state = State()
        new_state.data = np.copy(self.data)
        new_state.data[i, j] = symbol
        return new_state


def get_all_states_impl(current_state, current_symbol, all_states):
    for i in range(0, BOARD_ROWS):
        for j in range(0, BOARD_COLS):
            if current_state.data[i][j] == 0:
                newState = current_state.next_state(i, j, current_symbol)
                newHash = newState.hash()
                if newHash not in all_states.keys():
                    isEnd = newState.is_end()
                    all_states[newHash] = (newState, isEnd)
                    if not isEnd:
                        get_all_states_impl(newState, -current_symbol, all_states)

def get_all_states():
    current_symbol = 1
    current_state = State()
    all_states = dict()
    all_states[current_state.hash()] = (current_state, current_state.is_end())
    get_all_states_impl(current_state, current_symbol, all_states)
    return all_states

# all possible board configurations
all_states = get_all_states()


{0: (<__main__.State at 0x11145fef0>, False),
 6561: (<__main__.State at 0x111436e80>, False),
 10935: (<__main__.State at 0x111436eb8>, False),
 11664: (<__main__.State at 0x111436ef0>, False),
 12150: (<__main__.State at 0x111436f28>, False),
 12231: (<__main__.State at 0x111436f60>, False),
 12285: (<__main__.State at 0x111436518>, False),
 12294: (<__main__.State at 0x111436f98>, True),
 12288: (<__main__.State at 0x1114364a8>, False),
 12306: (<__main__.State at 0x10f75ac18>, False),
 12307: (<__main__.State at 0x10f75a208>, True),
 12290: (<__main__.State at 0x10f75a198>, False),
 12299: (<__main__.State at 0x10f75a128>, True),
 12286: (<__main__.State at 0x10f75a0b8>, True),
 12249: (<__main__.State at 0x10f75a048>, False),
 12276: (<__main__.State at 0x10f75ab70>, False),
 12282: (<__main__.State at 0x10f75ad30>, False),
 12283: (<__main__.State at 0x10f75ac88>, True),
 12278: (<__main__.State at 0x10f75ab38>, False),
 12281: (<__main__.State at 0x10f75aba8>, True),
 12252: (<_